<a href="https://colab.research.google.com/github/jeffheaton/app_deep_learning/blob/main/assignments/assignment_yourname_class4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# T81-558: Applications of Deep Neural Networks

- Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
- For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

**Module 4 Assignment: Classification and Regression Neural Network**

**Student Name: Your Name**


# Assignment Instructions

For this assignment, you will use the **crx.csv** dataset. This dataset is a public dataset that can you can find [here](https://archive.ics.uci.edu/ml/datasets/credit+approval). You should use the CSV file on my data site, at this location: [crx.csv](https://data.heatonresearch.com/data/t81-558/crx.csv) because it includes column headers. The primary use for this dataset is binary classification. There are 15 attributes, plus a target column that contains only + or -. Some of the columns have missing values.

You should train a neural network and return the predictions. You will submit these predictions to the **submit** function. See [Assignment #1](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/assignments/assignment_yourname_class1.ipynb) for details on how to submit an assignment or check that one was submitted.

Complete the following tasks:

- Your task is to replace missing values in columns _a2_ and _a14_ with values estimated by a neural network (one neural network for _a2_ and another for _a14_).
- Your submission file will contain the same headers as the source CSV: _a1_, _a2_, _s3_, _a4_, _a5_, _a6_, _a7_, _a8_, _a9_, _a10_, _a11_, _a12_, _a13_, _a14_, _a15_, and _a16_.
- You should only need to modify _a2_ and _a14_.
- Neural networks can be much more powerful at filling missing variables than median and mean.
- Train two neural networks to predict _a2_ and _a14_.
- The _y_ (target) for training the two nets will be _a2_ and _a14_, depending on which you are trying to fill.
- The x for training the two nets will be 's3','a8','a9','a10','a11','a12','a13','a15'. These are chosen because it is important not to use any columns with missing values; also, it could cause unwanted bias if we include the ultimate target (_a16_).
- ONLY predict new values for missing values in _a2_ and _a14_.
- You will likely get this small warning: Warning: The mean of column a14 differs from the solution file by 0.20238937709643778. (might not matter if small)


# Google CoLab Instructions

If you are using Google CoLab, it will be necessary to mount your GDrive so that you can send your notebook during the submit process. Running the following code will map your GDrive to `/content/drive`.


In [ ]:
try:
import copy
import torch
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
COLAB = True
print("Note: using Google CoLab")
%tensorflow_version 2.x
except:
    print("Note: not using Google CoLab")
    COLAB = False

# Early stopping (see module 3.4)


class EarlyStopping:
    def __init__(self, patience=5, min_delta=0, restore_best_weights=True):
        self.patience = patience
        self.min_delta = min_delta
        self.restore_best_weights = restore_best_weights
        self.best_model = None
        self.best_loss = None
        self.counter = 0
        self.status = ""

    def __call__(self, model, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_model = copy.deepcopy(model.state_dict())
        elif self.best_loss - val_loss >= self.min_delta:
            self.best_model = copy.deepcopy(model.state_dict())
            self.best_loss = val_loss
            self.counter = 0
            self.status = f"Improvement found, counter reset to {self.counter}"
        else:
            self.counter += 1
            self.status = f"No improvement in the last {self.counter} epochs"
            if self.counter >= self.patience:
                self.status = f"Early stopping triggered after {self.counter} epochs."
                if self.restore_best_weights:
                    model.load_state_dict(self.best_model)
                return True
        return False


# Make use of a GPU or MPS (Apple) if one is available.  (see module 3.2)

device = (
    "mps"
    if getattr(torch, "has_mps", False)
    else "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using device: {device}")


# Assignment Submit Function

You will submit the ten programming assignments electronically. The following **submit** function can be used to do this. My server will perform a basic check of each assignment and let you know if it sees any underlying problems.

**It is unlikely that should need to modify this function.**


In [1]:
import base64
import os
import numpy as np
import pandas as pd
import requests
import PIL
import PIL.Image
import io

# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - List of pandas dataframes or images.
# key - Your student key that was emailed to you.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.


def submit(data, key, no, source_file=None):
    if source_file is None and '__file__' not in globals():
        raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None:
        source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file:
        raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb', '.py']:
        raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    payload = []
    for item in data:
        if type(item) is PIL.Image.Image:
            buffered = BytesIO()
            item.save(buffered, format="PNG")
            payload.append({'PNG': base64.b64encode(
                buffered.getvalue()).decode('ascii')})
        elif type(item) is pd.core.frame.DataFrame:
            payload.append({'CSV': base64.b64encode(item.to_csv(
                index=False).encode('ascii')).decode("ascii")})
    r = requests.post("https://api.heatonresearch.com/assignment-submit",
                      headers={'x-api-key': key}, json={'payload': payload, 'assignment': no, 'ext': ext, 'py': encoded_python})
    if r.status_code == 200:
        print("Success: {}".format(r.text))
    else:
        print("Failure: {}".format(r.text))


# Assignment #4 Sample Code

The following code provides a starting point for this assignment.


In [2]:
import os
import pandas as pd
from scipy.stats import zscore
import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics

# This is your student key that I emailed to you at the beginnning of the semester.
# This is an example key and will not work.
key = "r2nrqz2pX53SGKnwA07UW52mBbNzuLpf8e2ZYIV9"


# You must also identify your source file.  (modify for your local setup)
# file='/content/drive/My Drive/Colab Notebooks/assignment_solution_class4.ipynb'  # Google CoLab
# file='C:\\Users\\jeffh\\projects\\t81_558_deep_learning\\assignments\\assignment_yourname_class4.ipynb'  # Windows
file = '/Users/zixuanni/Desktop/T81-558/assignment_Zixuan_class4.ipynb'  # Mac/Linux

# Begin assignment
df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/crx.csv", na_values=['?'])


In [11]:
import pandas as pd
import copy
import numpy as np
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import tqdm
from scipy.stats import zscore
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder

class EarlyStopping:
    def __init__(self, patience=5, min_delta=0, restore_best_weights=True):
        self.patience = patience
        self.min_delta = min_delta
        self.restore_best_weights = restore_best_weights
        self.best_model = None
        self.best_loss = None
        self.counter = 0
        self.status = ""

    def __call__(self, model, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_model = copy.deepcopy(model.state_dict())
        elif self.best_loss - val_loss >= self.min_delta:
            self.best_model = copy.deepcopy(model.state_dict())
            self.best_loss = val_loss
            self.counter = 0
            self.status = f"Improvement found, counter reset to {self.counter}"
        else:
            self.counter += 1
            self.status = f"No improvement in the last {self.counter} epochs"
            if self.counter >= self.patience:
                self.status = f"Early stopping triggered after {self.counter} epochs."
                if self.restore_best_weights:
                    model.load_state_dict(self.best_model)
                return True
        return False


def fill_missing_numeric(df, current, target):
    le = LabelEncoder()

    # Encode categorical variables
    original_df = df
    df_dummies = pd.get_dummies(
        df[['a1', 'a4', 'a5', 'a6', 'a7', 'a9', 'a10', 'a12', 'a13']]).astype(int)
    df = pd.concat([df, df_dummies], axis=1)

    train_df = df.dropna(subset=[current])
    pred_df = df[df[current].isnull()]

    x_columns = train_df.columns.drop(
        ['a1', 'a4', 'a5', 'a6', 'a7', 'a9', 'a10', 'a12', 'a13', 'a16', current])
    x_train = train_df[x_columns].values
    x_test = pred_df[x_columns].values
    y_train = train_df[current].values

    device = 'mps'

    x_train = torch.tensor(x_train, device=device, dtype=torch.float32)
    y_train = torch.tensor(y_train, device=device, dtype=torch.float32)
    x_test = torch.tensor(x_test, device=device, dtype=torch.float32)
    '''
    model = nn.Sequential(
        nn.Linear(x_train.shape[1], 50),
        nn.ReLU(),
        nn.Linear(50, 25),
        nn.ReLU(),
        nn.Linear(25, 1)
    ).to(device)

    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    model.train()
    for epoch in range(500):
        optimizer.zero_grad()
        out = model(x_train).flatten()
        loss = loss_fn(out, y_train)
        loss.backward()
        optimizer.step()

        if epoch % 100 == 0:
            print(f"Epoch {epoch}, loss: {loss.item()}")
    '''
    torch.manual_seed(42)

    # Cross-Validate
    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    # Early stopping parameters
    patience = 10

    fold = 0
    for train_idx, test_idx in kf.split(x_train):
        fold += 1
        print(f"Fold #{fold}")

        x_train, x_test = x_train[train_idx], x_train[test_idx]
        y_train, y_test = y_train[train_idx], y_train[test_idx]

    # PyTorch DataLoader
        train_dataset = TensorDataset(x_train, y_train)
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    # Create the model and optimizer
        model = nn.Sequential(
            nn.Linear(x_train.shape[1], 20),
            nn.ReLU(),
            nn.Linear(20, 10),
            nn.ReLU(),
            nn.Linear(10, 1)
        )
        model = torch.compile(model,backend="aot_eager").to(device)


        optimizer = optim.Adam(model.parameters(), lr=0.001)
        loss_fn = nn.MSELoss()

        # Early Stopping variables
        best_loss = float('inf')
        early_stopping_counter = 0

        # Training loop
        EPOCHS = 500
        epoch = 0
        done = False
        es = EarlyStopping()

        while not done and epoch<EPOCHS:
            epoch += 1
            model.train()
            for x_batch, y_batch in train_loader:
                optimizer.zero_grad()
                output = model(x_batch)
                loss = loss_fn(output, y_batch)
                loss.backward()
                optimizer.step()

        # Validation
        model.eval()
        with torch.no_grad():
            val_output = model(x_test)
            val_loss = loss_fn(val_output, y_test)

        if es(model, val_loss):
            done = True

    print(f"Epoch {epoch}/{EPOCHS}, Validation Loss: "
        f"{val_loss.item()}, {es.status}")

    model.eval()
    with torch.no_grad():
        pred = model(x_test)

    original_df.loc[original_df[current].isnull(
    ), current] = pred.cpu().numpy().flatten()

    return original_df


df_submit = fill_missing_numeric(df, 'a2', 'a16')
df_submit = fill_missing_numeric(df_submit, 'a14', 'a16')

submit(source_file=file, data=[df_submit], key=key, no=4)


Fold #1


/opt/homebrew/Caskroom/miniforge/base/envs/torch/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/opt/homebrew/Caskroom/miniforge/base/envs/torch/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/opt/homebrew/Caskroom/miniforge/base/envs/torch/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lea

KeyboardInterrupt: 

In [4]:
import numpy as np

# Given data
x1 = np.array([-1.2, -1.1, 0, 1.1, 1.2])
x2 = np.array([0.4, 0.3, 0.2, 0.1, 0])
y_actual = np.array([-1.5, -0.5, 0.3, 1.2, 1.9])

# Weight vector
w = np.array([0.3, 0.2, 0.1])

# Calculate predicted values ŷ for each data point
y_predicted = np.dot(np.vstack((np.ones(len(x1)), x1, x2)).T, w)
# Calculate squared errors (y - ŷ)^2
squared_errors = (y_actual - y_predicted)**2
# Calculate SSE by summing up squared errors
sse = np.sum(squared_errors)
print(sse)



5.231
